In [1]:
import copy

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
from matplotlib.widgets import Button, Slider
from tools.schemes import Comparator, K_gen, Solver, Sweeper, get_init

In [2]:
%matplotlib qt
custom_plots ={}

In [3]:
t = np.linspace(0, 0.001, 1000)

## Initiatiation


In [4]:
gen_inititiation_c = {'Q':0.002,'DH':0.5}

In [16]:
class K_init_full(K_gen):
    l = 1e9 * 0.00001  # 8 10
    l_ = 1e5  # 5 6

    qE = 1e9  # 8 10
    qE_ = 1e8  # 7 9
    H = 1e9  # 8 10
    diff= 1e8 # 8 9

    qH = 1e6  # 2000
    dQ = 1e9
    redQ = 1e3
    qQD = 2

    r = 1e8
    p = 1e-4
    rD_rec = 1e9
    rD_dis = 1e9
    D = 1
    D_ = 0.05

    qPh = 1e-4

def init_full_s(t, C, k=K_init_full()):
    [Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD] = C

    R1 = k.l * Q
    R2 = k.l_ * Qt
    R3 = k.diff * Qt * DH
    R4 = k.qE * Q_DH
    R5 = k.qE_ * Q_DHi
    # R6 = k.diff * Q_DHi
    R7 = k.H * Q_DHi
    R8 = k.diff * Q_DHb
    R9 = k.qH * Qt * QHH
    R10 = k.dQ * QH * QH
    R11 = k.redQ * Q * QHH
    R12 = k.qQD * Qt * QHD
    R13 = k.r * QH * D
    R14 = k.p * QHD
    R15 = k.rD_rec * D * D
    R16 = k.rD_dis * D * D
    R17 = k.D * Q * D
    R18 = k.D_ * QD
    R19 = k.qPh * Qt

    res = dict(
        dCl=-R1 + R2 + R10 - R11 - R17 + R18,
        dC2=R1 - R2 - R3 - R9 - R12 - R19,
        dC3=-R3 + R16,
        dC4=R3 - R4 + R5,
        dC5=R4 - R5  - R7, # - R6
        dC7=R7 - R8,
        dC8=R8 + 2 * R9 - 2 * R10 + 2 * R11 + R12 - R13,
        dC9=R8 - R13 - 2 * R15 - 2 * R16 - R17 + R18,
        dClO=-R9 + R10 - R11 + R14,
        dCll=-R12 + R13 - R14,
        dC12=R12 + R17 - R18,
    )

    return list(res.values())


init_full_i = get_init(
    '[Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD]',
    gen_inititiation_c
)

In [ ]:
class K_stricted(K_gen):
    l = 1e9 * 0.00001  # 8 10
    l_ = 1e5  # 5 6

    qE = 1e9  # 8 10
    H = 1e9  # 8 10
    diff= 1e8 # 8 9

    dQ = 1e9
    redQ = 1e3

    r = 1e8
    p = 1e-4
    rD_rec = 1e9
    rD_dis = 1e9

def stricted_s(t, C, k=K_stricted()):
    [Q, Qt, DH, QH, D, QHH] = C

    R1 = k.l * Q
    R2 = k.l_ * Qt
    R3 = k.H * Qt * DH
    R6 = k.qH * Qt * QHH
    R7 = k.dQ * QH * QH
    R13 = k.r * QH * D
    R15 = k.rD_rec * D * D

    res = dict(
        Q=-R1 + R2 + R7,
        Qt=R1 - R2 - R3 - R6,
        DH=-R3,
        QH=R3 + 2 * R6 - 2 * R7 - R13,
        D=R3 - R13 - 2 * R15,
        QHH=-R6 + R7,
    )

    return list(res.values())


stricted_i = get_init(
    '[Q, Qt, DH, QH, D, QHH]',
    gen_inititiation_c
)

In [21]:
class K_base(K_gen):
    l = K_init_full.l

    diff = K_init_full.diff

    dQ = K_init_full.dQ

    r = K_init_full.r
    rD_rec = K_init_full.rD_rec
    rD_dis = K_init_full.rD_dis


def init_base_s(t, C, k=K_base()):
    [Q, Qt, DH, QH, D, QHH] = C

    R1 = k.l * Q
    R2 = k.diff * Qt * DH
    R3 = k.r * QH * D
    R4 = k.dQ * QH * QH
    R5 = k.rD_rec * D * D
    R6 = k.rD_dis * D * D

    res = dict(
        Q=-R1 + R4,
        Qt=R1 - R2,
        DH=-R2 + R6,
        QH=R2 - R3 - 2 * R4,
        D=R2 - R3 - 2 * R5 - 2 * R6,
        QHH=R4,
    )

    return list(res.values())


init_base_i = get_init('[Q, Qt, DH, QH, D, QHH]', gen_inititiation_c)

## Polymerization only

In [ ]:
class K_old(K_gen):
    k = 1e4

def pol_old1_s(t, C, k=K_old()):
    [M] = C
    R1 = k.k * M

    res = dict(
        Q=-R1,
    )
    return list(res.values())


pol_old1_i = get_init('[M]', [0.9])

pol_old2_i = get_init('[M]', [0.9])


def pol_old2_s(t, C, k=K_old()):
    g = 4
    [M] = C
    R1 = g * k.k * M * (np.log(pol_old2_i['M'] / M)) ** ((g - 1) / g) if M > 1e-5 else 0

    res = dict(
        Q=-R1,
    )
    return list(res.values())

In [34]:
class K_pol_full(K_gen):
    init = 1e3
    prop = 1e3  # 2 4
    trans_sol = 5
    trans_m = 1e-3  # -3 0
    inh = 1e2  # 2 3
    ter_lin = 1e7
    ter_rec = 1e7
    ter_dis = 1e7


def pol_full_s(t, C, k=K_pol_full()):
    [D, M, PR, MR, Sol, Q] = C
    R1 = k.init * D * M
    R2 = k.prop * PR * M
    R3 = k.prop * MR * M
    R4 = k.trans_sol * PR * Sol
    R5 = k.trans_m * PR * M
    R6 = k.inh * PR * Q
    R7 = k.ter_lin * PR
    R8 = k.ter_rec * PR * PR
    R9 = k.ter_dis * PR * PR

    res = dict(
        D=-R1+0.0001,
        M=-R1 - R2 - R3,
        PR=R1 + R3 - R4 - R5 - R6 - R7 - 2 * R8 - 2 * R9,
        MR=-R3 + R5,
        Sol=-R4 - R5,
        Q=-R6,
    )

    return list(res.values())


pol_full_i = get_init(
    '[D, M, PR, MR, Sol, Q]',
    {'D': 0.0001, 'M': 2, 'Q': 0.02},
)

## Polymerization + initiation

In [ ]:
class K_mix_full(K_gen):
    l = 1e9 * 0.00001  # 8 10
    l_ = 1e5  # 5 6

    qE = 1e9  # 8 10
    qE_ = 1e8  # 7 9
    H = 1e9  # 8 10
    diff= 1e8 # 8 9

    qH = 1e6  # 2000
    dQ = 1e9
    redQ = 1e3
    qQD = 2

    r = 1e8
    p = 1e-4*1e5
    rD_rec = 1e9
    rD_dis = 1e9
    D = 1
    D_ = 0.05

    qPh = 1e-4

    init = 1e3
    prop = 1e3  # 2 4
    trans_sol = 5
    trans_m = 1e-3  # -3 0
    inh = 1e2  # 2 3
    ter_lin = 1e7
    ter_rec = 1e7
    ter_dis = 1e7


def mix_full_s(t, C, k=K_mix_full()):
    [Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD, M, PR, MR, Sol] = C

    R1 = k.l * Q
    R2 = k.l_ * Qt
    R3 = k.diff * Qt * DH
    R4 = k.qE * Q_DH
    R5 = k.qE_ * Q_DHi
    R6 = k.diff * Q_DHi
    R7 = k.H * Q_DHi
    R8 = k.diff * Q_DHb
    R9 = k.qH * Qt * QHH
    R10 = k.dQ * QH * QH
    R11 = k.redQ * Q * QHH
    R12 = k.qQD * Qt * QHD
    R13 = k.r * QH * D
    R14 = k.p * QHD
    R15 = k.rD_rec * D * D
    R16 = k.rD_dis * D * D
    R17 = k.D * Q * D
    R18 = k.D_ * QD
    R19 = k.qPh * Qt

    R1P = k.init * D * M
    R2P = k.prop * PR * M
    R3P = k.prop * MR * M
    R4P = k.trans_sol * PR * Sol
    R5P = k.trans_m * PR * M
    R6P = k.inh * PR * Q
    R7P = k.ter_lin * PR
    R8P = k.ter_rec * PR * PR
    R9P = k.ter_dis * PR * PR

    res = dict(
        Q=-R1 + R2 + R10 - R11 - R17 + R18-R6P,
        Qt=R1 - R2 - R3 - R9 - R12 - R19,
        DH=-R3 + R16,
        Q_DH=R3 - R4 + R5,
        Q_DHi=R4 - R5 - R6 - R7,
        Q_DHb=R7 - R8,
        QH=R8 + 2 * R9 - 2 * R10 + 2 * R11 + R12 - R13,
        D=R8 - R13 - 2 * R15 - 2 * R16 - R17 + R18 -R1P,
        QHH=-R9 + R10 - R11 + R14,
        QHD=-R12 + R13 - R14,
        QD=R12 + R17 - R18,
        M=-R1P - R2P - R3P,
        PR=R1P + R3P - R4P - R5P - R6P - R7P - 2 * R8P - 2 * R9P,
        MR=-R3P + R5P,
        Sol=-R4P - R5P,
    )

    return list(res.values())


mix_full_i = get_init(
    '[Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD, M, PR, MR, Sol]',
    {'Q':0.001,'DH':0.001,'M':2}
)

In [ ]:
class K_mix_base(K_gen):
    # INITIATION
    l = K_init_full.l

    H2 = K_init_full.diff

    dQ = K_init_full.dQ

    r2 = K_init_full.r
    rD_rec = K_init_full.rD_rec
    rD_dis = K_init_full.rD_dis

    # POLYMERIZATION
    pI = 0
    pP = 1e4
    pDD = 0
    pDrP = 0
    pDdP = 0

def pol_base_s(t, C, k=K_pol_full()):
    [Q, Qt, DH, QH, D, QHH, M, DM, P] = C

    R1 = k.l * Q
    R2 = k.l_ * Qt
    R3 = k.H * Qt * DH
    R4 = k.qH * Qt * QHH
    R5 = k.redQ * QH * QH
    R6 = k.r * QH * D
    R7 = k.rD * D * D
    R8 = k.pI * D * M
    R9 = k.pP * DM * M
    R10 = k.pP * P * M
    R11 = k.pDD * P * D
    R12 = k.pDrP * P * P
    R13 = k.pDdP * P * P

    res = dict(
        Q=-R1 + R2 + R5,
        Qt=R1 - R2 - R3 - R4,
        DH=-R3,
        QH=R3 + 2 * R4 - 2 * R5 - R6,
        D=R3 - R6 - 2 * R7 - R8 - R11,
        QHH=-R4 + R5,
        M=-R8 - R9 - R10,
        DM=R8 - R9,
        P=R9 - R11 - 2 * R12 - 2 * R13,
    )
    return list(res.values())


pol_base_i = get_init(
    '[Q, Qt, DH, QH, D, QHH, M, DM, P]',
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
)

In [ ]:
class K_simple(K_gen):
    # INITIATION
    i = 1e4

    p = 1e5
    l = 10
    q = 10

def pol_simple_s(t, C, k=K_simple()):
    [Q, D, M, cP] = C
    R1 = k.i * Q
    R2 = k.p * D * M
    R3 = k.p * cP * M
    R4 = k.l * cP
    R5 = k.q * cP * cP

    res = dict(
        Q=-R1,
        D=R1 - R2,
        M=-R2 - R3,
        P=R2 - R4 - 2 * R5,
    )
    return list(res.values())


pol_simple_i = get_init('[Q, D, M, cP]', [0.001, 0, 0.9, 0])

# Examples

In [6]:
# Initiation full
Y = Solver(init_full_s, K_init_full(), init_full_i,  t)
# Y = Sweeper(Solver(full_s, K(), full_i,  t),Solver(base_s, K(), base_i,  t))
LIM = None
AUTO = False
compounds = dict(
    Q=0,
    # DH=0,
    QHH=0,
    QH=2,
    D=1,
    QHD=2,
)

In [22]:
# Initiation base
# Y = Solver(init_base_s, K_base(), init_base_i, t)
Y = Sweeper(
    Solver(init_full_s, K_init_full(), init_full_i, t),
    Solver(init_base_s, K_base(), init_base_i, t),
)

# Y = Sweeper(
#     Solver(init_full_s, K_init_full(), init_full_i, t),
#     Solver(init_full_s, K_init_full(), init_full_i, t),
# )

LIM = None
AUTO = True
compounds = dict(
    Q=0,
    # DH=0,
    QHH=0,
    QH=2,
    D=1,
)

In [35]:
# Polimerization full
Y = Solver(pol_full_s, K_pol_full(), pol_full_i,  t)
LIM = None
AUTO = False

def a_plot():
    return (1 - Y['M'] / Y.initial['M']) * 100

custom_plots = dict(
    conv=a_plot,
)

compounds = dict(
    Q=1,
    M=0,
    # Sol=1,
    MR=1,
    D=1,
)

In [ ]:
Y = Solver(mix_full_s, K_mix_full(), mix_full_i, t)
LIM = None
AUTO = False
compounds = dict(
    M=0,
    Q=0,
    DH=0,
    D=1,
    QHH=0,
    QH=1,
    # M=0,
)


def plot1():
    return (1 - Y['M'] / Y.initial['M']) * 100


def plot2():
    return (1 - Y.y1['M'] / Y.y1.initial['M']) * 100 - (1 - Y.y2['M'] / Y.y2.initial['M']) * 100


custom_plots = dict(
    conv=plot1,
)

## View

In [36]:
# Plots
fig, ax = plt.subplots()
gs = plt.GridSpec(2, 2, figure=fig)
fig.subplots_adjust(left=0.25, right=0.99, bottom=0.05, top=0.99, hspace=0.1, wspace=0.1)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
scal = FuncFormatter(lambda x, pos: f'{x*1000: .2f}')

K_stricted = Y.K


def get_ax(gs, keys_num):
    ax = plt.subplot(gs)

    for c in [key for key, value in compounds.items() if value == keys_num]:
        plots[c] = ax.plot(Y.T, Y[c], label=c)[0]
    if keys_num == 2:
        for c, func_plot in custom_plots.items():
            plots[c] = ax.plot(
                Y.T,
                func_plot(),
                label=c,
            )[0]

    ax.legend()
    ax.xaxis.set_major_formatter(scal)
    if LIM is not None:
        ax.set_ylim([-LIM, LIM])

    return ax


# Plots
plots = {}
ax0 = get_ax(gs=gs[0, 0:2], keys_num=0)
ax1 = get_ax(gs=gs[1, 0], keys_num=1)
ax2 = get_ax(gs=gs[1, 1], keys_num=2)

# Sliders
sliders = {}
buttons = {}
K_new = K_stricted.copy()


def resolve(event=None):
    Y.solve(K=K_new)
    for c, plot in plots.items():
        if c in custom_plots:
            plot.set_ydata(custom_plots[c]()),
            continue
        plot.set_ydata(Y[c])
    fig.canvas.draw_idle()


def get_slider_action(k):
    def update(val):
        K_new[k] = K_stricted[k] * 10**val
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        if AUTO:
            resolve()

    return update


def get_zero_button_action(k):
    def update(event):
        K_new[k] = 0
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        fig.canvas.draw_idle()
        if AUTO:
            resolve()

    return update


def reset(event):
    global K_new
    K_new = copy.deepcopy(K_stricted)
    for k in sliders:
        sliders[k].reset()
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
    fig.canvas.draw_idle()
    if AUTO:
        resolve()


solve_axes = fig.add_axes([0.02, 0.05, 0.05, 0.05])
solve_button = Button(solve_axes, 'Solve', hovercolor='0.975')
solve_button.on_clicked(resolve)

reset_axes = fig.add_axes([0.08, 0.05, 0.05, 0.05])
reset_button = Button(reset_axes, 'Reset', hovercolor='0.975')
reset_button.on_clicked(reset)

for i, k in enumerate(K_stricted):
    k_axes = fig.add_axes(
        [
            0.03,  # left
            0.95 - 0.04 * i,  # bottom
            0.10,  # width
            0.03,  # height
        ]
    )
    amp_slider = Slider(
        ax=k_axes,
        label=k,
        valmin=-3,
        valmax=3,
        valinit=0,
        valstep=0.1,
        orientation="horizontal",
    )
    sliders[k] = amp_slider
    amp_slider.on_changed(get_slider_action(k))

    k_zero_axes = fig.add_axes(
        [
            0.15,  # left
            0.955 - 0.04 * i,  # bottom
            0.05,  # width
            0.02,  # height
        ]
    )
    zero_button = Button(k_zero_axes, f"{K_new[k]: .1e}", hovercolor='0.975')
    buttons[k] = zero_button
    zero_button.on_clicked(get_zero_button_action(k))